# Dataset Preprocessing

This notebook is dedicated to preprocess the given dataset into a required format/structure and divide it into train/val/test splits.


# Google Colab only

### Download required packages

In [ ]:
!pip install -r https://raw.githubusercontent.com/pmalesa/lesion_detector/main/notebooks/requirements.txt

### Mount DeepLesion images and checkpoints from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content

# remove existing link if any
!rm -rf data/deeplesion
!rm -rf faster_rcnn_checkpoints

!mkdir -p data
!ln -s /content/drive/MyDrive/deeplesion/data/deeplesion data/deeplesion
!ls -l data

# Import all packages

In [ ]:
# General packages
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy.typing import NDArray
from typing import Any
from PIL import Image
import shutil
import random
from pathlib import Path
import yaml
import cv2

# Image preprocessing and utility methods

In [ ]:
def load_metadata(path) -> pd.DataFrame:
    """
    Loads metadata from the given path and
    returns it as a pandas DataFrame.
    """
    
    return pd.read_csv(path)

def normalize(img: NDArray[np.uint16], per_image_norm: bool):
    """
    Normalizes the input image
    """

    img = img.astype(np.float32)
    if not per_image_norm:
        return img / 65535.0
    max = np.max(img)
    min = np.min(img)
    img = (img - min) / (max - min)
    return img

def convert_to_hu(img: NDArray[np.uint16], norm: bool, hu_min=-1024, hu_max=3071):
    """
    Converts the pixel data of a uint16
    CT image to Hounsfield Units (HU).
    """

    hu_img = img.astype(np.int32) - 32768
    hu_img = np.clip(hu_img, hu_min, hu_max).astype(np.float32)
    if norm:
        hu_img = (hu_img - hu_min) / (hu_max - hu_min)
        hu_img = np.clip(hu_img, 0.0, 1.0)
    return hu_img

def load_image(path: str, hu_scale: bool = True, norm: bool = True, per_image_norm: bool = True):
    """
    Loads an image given its path
    and returns it as a numpy array.
    """
    
    img = Image.open(path)
    img_array = np.array(img)
    if hu_scale:
        hu_min = -160
        hu_max = 240
        return convert_to_hu(img_array, norm, hu_min, hu_max)
    elif norm:
        return normalize(img_array, per_image_norm)
    return img_array

def save_image(img_array: NDArray[Any], path: str):
    """
    Saves the input image to the specified path.
    """

    if img_array.dtype != np.uint16:
        img_array = img_array.astype(np.uint16)
    img = Image.fromarray(img_array)
    img.save(path)

def save_normalized_image_uint8(img_array: NDArray[Any], path: str):
    """
    Saves the normalized input image data (0.0 - 1.0) 
    to the specified path with uint8 precision (0 - 255).
    """

    img_array_scaled = (img_array * 255.0).clip(0, 255).astype(np.uint8)
    img = Image.fromarray(img_array_scaled)
    img.save(path)

def show_image(img: NDArray[np.float32], title="Example Image", cmap="gray"):
    """
    Shows the image given its data, title and colour map.
    """

    plt.figure(figsize=(5, 5))
    plt.imshow(img, cmap=cmap)
    if title is not None:
        plt.title(title)
    plt.axis("off")
    plt.show()


# Set paths to DeepLesion images and metadata

## Paths to unprocessed data

In [ ]:
# Google Colab
deeplesion_metadata_path = Path("data/deeplesion/deeplesion_metadata.csv")
deeplesion_image_path = Path("data/deeplesion/key_slices/")

In [ ]:
# Local
deeplesion_metadata_path = Path("../data/deeplesion_metadata.csv")
deeplesion_image_path = Path("../data/deeplesion/key_slices/")

## Paths to processed data

In [ ]:
deeplesion_data_dir = Path("data/deeplesion/")
deeplesion_preprocessed_image_path = deeplesion_data_dir / "deeplesion_preprocessed_uint8/key_slices"
deeplesion_preprocessed_metadata_path = deeplesion_data_dir / "deeplesion_preprocessed_uint8/deeplesion_metadata_preprocessed.csv"

# Path to deeplesion metadata in COCO format
deeplesion_coco_json_path = deeplesion_data_dir / "deeplesion_coco.json"

# Preprocess the DeepLesion dataset

In [ ]:
'''
Preprocess the DeepLesion dataset by:
    - Choosing only images that come from the original validation and test set divisions (only these have lesion type annotations),
    - Normalizing each image according to a fixed HU scale window [-160, 240] (can be also [-1024, 3071]),
    - Resizing each image to 512x512 if necessary (and adjusting the target bounding box coordinates),
    - Saving the new metadata file with resized images and adjusted target bounding boxes. 
'''

deeplesion_metadata = load_metadata(deeplesion_metadata_path)

# Clear the existing directory with preprocessed images
if deeplesion_preprocessed_image_path.exists() and deeplesion_preprocessed_image_path.is_dir():
    shutil.rmtree(deeplesion_preprocessed_image_path)
deeplesion_preprocessed_image_path.mkdir(parents=True, exist_ok=True)

for idx, row in deeplesion_metadata.iterrows():
    # Extract only images with annotated lesions (Val + Test)
    if row["Train_Val_Test"] == 1:
        continue

    file_name = row["File_name"]
    bbox_str = row["Bounding_boxes"]
    size_str = row["Image_size"]
    image_path = os.path.join(deeplesion_image_path, file_name)
    preprocessed_image_path = os.path.join(deeplesion_preprocessed_image_path, file_name)
    image_data = load_image(image_path)

    # Extract ground truth bounding box' coordinates
    bbox_coords = [float(val) for val in bbox_str.split(",")]
    x1, y1, x2, y2 = [round(c) for c in bbox_coords]

    # Extract sizes
    image_sizes = [int(val) for val in size_str.split(",")]
    width, height = [size for size in image_sizes]

    # Rescale to (512 x 512) if necessary
    if (width, height) != (512, 512):
        image_data = cv2.resize(
            image_data, (512, 512), interpolation=cv2.INTER_AREA
        )
        scale_x = 512 / width
        scale_y = 512 / height
        x1 = round(x1 * scale_x)
        y1 = round(y1 * scale_y)
        x2 = round(x2 * scale_x)
        y2 = round(y2 * scale_y)
        height = 512
        width = 512

        deeplesion_metadata.at[idx, "Bounding_boxes"] = f"{x1}, {y1}, {x2}, {y2}"
        deeplesion_metadata.at[idx, "Image_size"] = "512, 512"

    save_normalized_image_uint8(image_data, preprocessed_image_path)    

# Save preprocessed metadata csv file
deeplesion_metadata.to_csv(deeplesion_preprocessed_metadata_path)

# Verify if all images have 512x512 size
deeplesion_metadata_preprocessed = load_metadata(deeplesion_preprocessed_metadata_path)
for idx, row in deeplesion_metadata_preprocessed.iterrows():
    if row["Train_Val_Test"] == 1:
        continue
    size_str = row["Image_size"]
    image_sizes = [int(val) for val in size_str.split(",")]
    width, height = [size for size in image_sizes]
    if (width, height) != (512, 512):
        raise ValueError(f"ERROR: Not all images have required size!")
print(f"SUCCES: All images were preprocessed correctly.")
    

# Convert DeepLesion metadata to COCO format

In [ ]:
deeplesion_metadata_preprocessed = load_metadata(deeplesion_preprocessed_metadata_path)

images = []
annotations = []
categories = [
    {"id": 1, "name": "bone"},
    {"id": 2, "name": "abdomen"},
    {"id": 3, "name": "mediastinum"},
    {"id": 4, "name": "liver"},
    {"id": 5, "name": "lung"},
    {"id": 6, "name": "kidney"},
    {"id": 7, "name": "soft tissue"},
    {"id": 8, "name": "pelvis"}
]

image_counter = 1
annotation_id = 1
image_id_map = {}

for idx, row in deeplesion_metadata_preprocessed.iterrows():
    # Extract only images with annotated lesions (Val + Test)
    if row["Train_Val_Test"] == 1:
        continue

    file_name = row["File_name"]
    lesion_type = row["Coarse_lesion_type"]
    bbox_str = row["Bounding_boxes"]
    size_str = row["Image_size"]

    # Extract ground truth bounding box' coordinates
    bbox_coords = [float(val) for val in bbox_str.split(",")]
    x1, y1, x2, y2 = [round(c) for c in bbox_coords]

    # Extract sizes
    image_sizes = [int(val) for val in size_str.split(",")]
    width, height = [size for size in image_sizes]

    # Register image
    if file_name not in image_id_map:
        image_id_map[file_name] = image_counter
        images.append({
            "id": image_counter,
            "file_name": file_name,
            "width": width,
            "height": height
        })
        image_counter += 1
    image_id = image_id_map[file_name]

    # Initialize ground truth bounding box' parameters
    bbox = [x1, y1, x2 - x1, y2 - y1]
    area = bbox[2] * bbox[3]

    annotations.append({
        "id": annotation_id,
        "image_id": image_id,
        "category": lesion_type,
        "bbox": bbox,
        "area": area,
        "iscrowd": 0    # Normal object (not crowd of indistinct objects, that can't be cleanly separated)
    })

    annotation_id += 1

# Save to JSON
deeplesion_coco_format = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}

with open(deeplesion_coco_json_path, "w") as f:
    json.dump(deeplesion_coco_format, f, indent=2)


# Convert DeepLesion dataset

## Set seed

In [ ]:
# Change this seed to create different train/val/test splits (42, 314, 666)
current_seed = 42

## YOLO format

In [ ]:
deeplesion_metadata_preprocessed = load_metadata(deeplesion_preprocessed_metadata_path)

# Source directories
image_dir = deeplesion_preprocessed_image_path
label_dir = deeplesion_data_dir / "labels_unsorted"

# Create .txt files
label_dir.mkdir(parents=True, exist_ok=True)

# Create target directory
target_dir = deeplesion_data_dir / "deeplesion_yolo"
if target_dir.exists() and target_dir.is_dir():
    shutil.rmtree(target_dir)

for idx, row in deeplesion_metadata_preprocessed.iterrows():
    # Extract only images with annotated lesions (Val + Test)
    if row["Train_Val_Test"] == 1:
        continue

    file_name = row["File_name"]
    image_path = os.path.join(str(image_dir), file_name)
    label_path = os.path.join(str(label_dir), file_name.replace(".png", ".txt"))

    if not os.path.exists(image_path):
        continue

    lesion_type = row["Coarse_lesion_type"] - 1 # YOLOv5 requires class IDs starting at 0
    bbox_str = row["Bounding_boxes"]
    size_str = row["Image_size"]

    # Extract ground truth bounding box' coordinates
    bbox_coords = [float(val) for val in bbox_str.split(",")]
    x1, y1, x2, y2 = [round(c) for c in bbox_coords]

    # Extract sizes
    image_sizes = [int(val) for val in size_str.split(",")]
    width, height = [size for size in image_sizes]

    bbox_width = x2 - x1
    bbox_height = y2 - y1
    x_center = x1 + bbox_width / 2
    y_center = y1 + bbox_height / 2

    # Normalize
    x_center /= width
    y_center /= height
    bbox_width /= width
    bbox_height /= height

    with open(label_path, 'a') as f:
        f.write(f"{lesion_type} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

# -----------------------------------------------------------------------------

splits = ["train", "val", "test"]
for split in splits:
    (target_dir / "images" / split).mkdir(parents=True, exist_ok=True)
    (target_dir / "labels" / split).mkdir(parents=True, exist_ok=True)

# Collect all images with annotated lesions
annotated_images = [img for img in image_dir.glob("*.png") if (label_dir / (img.stem + ".txt")).exists()]
random.seed(current_seed)
random.shuffle(annotated_images)

# Split into train, val and test sets
n_total = len(annotated_images)
n_train = int(0.7 * n_total)
n_val = int(0.15 * n_total)

train_images = annotated_images[:n_train]
val_images = annotated_images[n_train:n_train + n_val]
test_images = annotated_images[n_train + n_val:]

splits_map = {
    "train": train_images,
    "val": val_images,
    "test": test_images
}

# Copy image files
for split, images in splits_map.items():
    for image_path in images:
        label_path = label_dir / (image_path.stem + ".txt")
        shutil.copy(image_path, target_dir / "images" / split / image_path.name)
        shutil.copy(label_path, target_dir / "labels" / split / label_path.name)

print(f"Split done! Total = {n_total}")

# Generate deeplesion.yaml
dataset_root = os.path.abspath(str(target_dir))
deeplesion_yaml = {
    "path": dataset_root,
    "train": os.path.join(dataset_root, "images/train"),
    "val": os.path.join(dataset_root, "images/val"),
    "test": os.path.join(dataset_root, "images/test"),
    "nc": 8,
    "names": [
        "bone",
        "abdomen",
        "mediastinum",
        "liver",
        "lung",
        "kidney",
        "soft_tissue",
        "pelvis"
    ]
}

with open(target_dir / "deeplesion.yaml", "w") as f:
    yaml.dump(deeplesion_yaml, f)

# Remove directory with unsorted labels
if label_dir.exists() and label_dir.is_dir():
    shutil.rmtree(label_dir)


## Faster R-CNN format

In [ ]:
deeplesion_metadata_preprocessed = load_metadata(deeplesion_preprocessed_metadata_path)

# Source directories
image_dir = deeplesion_preprocessed_image_path
label_dir = deeplesion_data_dir / "labels_unsorted"

# Create .txt files
label_dir.mkdir(parents=True, exist_ok=True)

# Create target directory
target_dir = deeplesion_data_dir / "deeplesion_fasterrcnn_split_X"  # "X" is to be replaced with a number 1-3
if target_dir.exists() and target_dir.is_dir():
    shutil.rmtree(target_dir)

for idx, row in deeplesion_metadata_preprocessed.iterrows():
    # Extract only images with annotated lesions (Val + Test)
    if row["Train_Val_Test"] == 1:
        continue

    file_name = row["File_name"]
    image_path = os.path.join(str(image_dir), file_name)
    label_path = os.path.join(str(label_dir), file_name.replace(".png", ".txt"))

    if not os.path.exists(image_path):
        continue

    lesion_type = row["Coarse_lesion_type"] # Faster R-CNN uses the class 0 implicitly as the background class (no need for subtraction)
    bbox_str = row["Bounding_boxes"]
    size_str = row["Image_size"]

    # Extract ground truth bounding box' coordinates
    bbox_coords = [float(val) for val in bbox_str.split(",")]
    x1, y1, x2, y2 = [round(c) for c in bbox_coords]

    with open(label_path, 'a') as f:
        f.write(f"{lesion_type} {x1} {y1} {x2} {y2}\n")

# -----------------------------------------------------------------------------

splits = ["train", "val", "test"]
for split in splits:
    (target_dir / "images" / split).mkdir(parents=True, exist_ok=True)
    (target_dir / "labels" / split).mkdir(parents=True, exist_ok=True)

# Collect all images with annotated lesions
annotated_images = [img for img in image_dir.glob("*.png") if (label_dir / (img.stem + ".txt")).exists()]
random.seed(current_seed)
random.shuffle(annotated_images)

# Split into train, val and test sets
n_total = len(annotated_images)
n_train = int(0.7 * n_total)
n_val = int(0.15 * n_total)

train_images = annotated_images[:n_train]
val_images = annotated_images[n_train:n_train + n_val]
test_images = annotated_images[n_train + n_val:]

splits_map = {
    "train": train_images,
    "val": val_images,
    "test": test_images
}

# Copy image files
for split, images in splits_map.items():
    for image_path in images:
        label_path = label_dir / (image_path.stem + ".txt")
        shutil.copy(image_path, target_dir / "images" / split / image_path.name)
        shutil.copy(label_path, target_dir / "labels" / split / label_path.name)

print(f"Split done! Total = {n_total}")

# Remove directory with unsorted labels
if label_dir.exists() and label_dir.is_dir():
    shutil.rmtree(label_dir)
